# Parameters 

# Import

In [1]:
import torch
import numpy as np
import random
import pandas as pd
import gc
from IPython.display import display, HTML
from pathlib import Path
from datetime import datetime
from transformers import BertTokenizerFast, DebertaTokenizerFast

In [2]:
def import_fold(path, fold):
    train = pd.read_csv(f"{path}/train.{fold}.csv")
    test = pd.read_csv(f"{path}/test.{fold}.csv")
    train_origin = train["origin"].tolist()
    train_target = train["target"].tolist()
    train_labels = train["label"].tolist()
    test_origin = test["origin"].tolist()
    test_target = test["target"].tolist()
    test_labels = test["label"].tolist()
    return train_origin, train_target, train_labels, test_origin, test_target, test_labels

## Dataset

In [3]:
class SemanticDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

## Text Segment Input Strategies
Different Input Strategies used for Language Model Input. Input of Language Model is pair ´(A,B)´ with ´A´ and ´B´ of type:
1. Sentence (S)
2. Title + Sentence (TS)
3. Title + Sentence + Date (TSD)
4. Sentence + Title + Date (STD)

*Sentence* is the extracted Sentence from News Article, *Title* is the News Article Title and *Date* is the publishing date of the article

## Model Settings

In [4]:
input_strategies = [4,3,2,1]
subset_labels_types = [True, False]

In [5]:
# Model
model_checkpoints = ['microsoft/deberta-base', 'bert-base-uncased', 'bert-base-cased', 'bert-large-uncased', 'bert-large-cased']
batch_size = 8
metric_name = "accuracy"
num_epoch = 10

# Fold
num_folds = 4


In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

args = TrainingArguments(
    "semantic-test",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

### Helper functions

In [7]:
from sklearn.metrics import classification_report
import collections

#classification_threshold = 0.

def flatten(d, parent_key='', sep='__'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [8]:
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

# Loop

In [ ]:
for model_checkpoint in model_checkpoints:
    for input_strategy in input_strategies:
        for subset_labels in subset_labels_types:            

            labels = ["none", "attribution", "causal", "conditional", "contrast", "description", "equivalence", "fulfillment", "identity", "purpose", "summary", "temporal"]

            # Use subset of labels (defined below)
            if subset_labels:
                labels = ["none", "causal", "contrast", "equivalence", "identity", "temporal", "others"]

            # Import Paths
            import_path = f"data/export/{input_strategy}"
            if subset_labels:
                import_path = f"data/export_subset/{input_strategy}"

            # Export Paths
            export_path_model = "data/model"
            export_path_eval = "data/eval"

            if subset_labels:
                export_path_model += "_subset"
                export_path_eval += "_subset"

            # Metrics
            def compute_metrics(eval_pred):
                global labels
                predictions, true_labels = eval_pred
                # take most probable guess
                predictions = np.argmax(predictions, axis=-1)
                return flatten(classification_report(
                    y_true=true_labels,
                    y_pred=predictions,
                    target_names=labels,
                    zero_division=0,
                    output_dict=True))

            # Tokenizer
            if "microsoft" in model_checkpoint:
                tokenizer = DebertaTokenizerFast.from_pretrained(model_checkpoint)
            else:
                tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

            # Train
            result = []
            num_labels = len(labels)
            count = 0
            for i in range(num_folds):
                model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
                # import Fold data
                train_origin, train_target, train_labels, test_origin, test_target, test_labels = import_fold(import_path, i)
                # tokenize
                train_encodings = tokenizer(train_origin, train_target, truncation=True, padding=True, return_token_type_ids=True)
                test_encodings = tokenizer(test_origin, test_target, truncation=True, padding=True, return_token_type_ids=True)
                # dataset creation
                train_dataset = SemanticDataset(train_encodings, train_labels)
                test_dataset = SemanticDataset(test_encodings, test_labels)
                # create Trainer
                trainer = Trainer(
                    model,
                    args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    tokenizer=tokenizer,
                    compute_metrics=compute_metrics
                )
                # train & evaluate
                trainer.train()
                ev = trainer.evaluate(test_dataset)
                acc = ev["eval_accuracy"]
                print(f"Accuracy: {acc}")
                result.append(ev)
                # Save Model
                path = f"{export_path_model}/{input_strategy}/{model_checkpoint.replace(r'/', '-')}/epoch_{num_epoch}/fold_{count}"
                # Check if Path exists
                Path(path).mkdir(parents=True, exist_ok=True)
                trainer.save_model(path)
                # Free memory
                del model, train_origin, train_target, train_labels, test_origin, test_target, test_labels, train_encodings, test_encodings, train_dataset, test_dataset, trainer 
                gc.collect()
                torch.cuda.empty_cache()
                count += 1

            # Calc Metrics
            def transform_to_regular_dict(result):
                output_dict = {}
                count = 0
                for eval_item in result:
                    for key in eval_item:
                        if count == 0:
                          output_dict[key] = [float(eval_item[key])]
                        else:
                          output_dict[key].append(eval_item[key]) 
                    count += 1
                return output_dict

            eval_dict = transform_to_regular_dict(result)
            eval_df = pd.DataFrame(eval_dict)

            def add_mean_std_row(df):
                row_mean = []
                row_std = []
                for column in df:
                    row_mean.append(mean(df[column]))
                    row_std.append(stddev(df[column], ddof=1))
                df = df.append(pd.DataFrame([row_mean], columns=df.columns), ignore_index=True)
                df = df.append(pd.DataFrame([row_std], columns=df.columns), ignore_index=True)
                # add better readable Index
                df["fold"] = ["1", "2", "3", "4", "avg", "std"]
                df = df.set_index("fold")
                return df

            eval_df = add_mean_std_row(eval_df)
            display(HTML(eval_df.to_html()))

            # Eval
            # Check if Path exists
            path = f"{export_path_eval}/{input_strategy}/"
            Path(path).mkdir(parents=True, exist_ok=True)
            eval_df.to_csv(f"{path}/{model_checkpoint.replace(r'/', '-')}_epoch_{num_epoch}.csv")

            # datetime object containing current date and time
            now = datetime.now()

            # LOGS
            # Open the file in append & read mode ('a+')
            with open("logs.txt", "a+") as file_object:
                # Move read cursor to the start of file.
                file_object.seek(0)
                # If file is not empty then append '\n'
                data = file_object.read(100)
                if len(data) > 0 :
                    file_object.write("\n")
                # Append text at the end of file
                file_object.write(f"{now} - finished: {model_checkpoint}, Batch: {batch_size}, Epochs: {num_epoch}, Folds: {num_folds}, Subset: {subset_labels}, Input_strategy: {input_strategy}")

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'config', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias',

Epoch,Training Loss,Validation Loss,None Precision,None Recall,None F1-score,None Support,Causal Precision,Causal Recall,Causal F1-score,Causal Support,Contrast Precision,Contrast Recall,Contrast F1-score,Contrast Support,Equivalence Precision,Equivalence Recall,Equivalence F1-score,Equivalence Support,Identity Precision,Identity Recall,Identity F1-score,Identity Support,Temporal Precision,Temporal Recall,Temporal F1-score,Temporal Support,Others Precision,Others Recall,Others F1-score,Others Support,Accuracy,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support
1,No log,0.999048,0.802817,0.861461,0.831106,794,0.185567,0.165138,0.174757,109,0.000000,0.000000,0.000000,41,0.235060,0.694118,0.351190,85,0.000000,0.000000,0.000000,20,0.392157,0.136054,0.202020,147,0.000000,0.000000,0.000000,55,0.624301,0.230800,0.265253,0.222725,1251,0.587762,0.624301,0.590323,1251
2,No log,0.876993,0.786441,0.876574,0.829065,794,0.359375,0.211009,0.265896,109,0.666667,0.048780,0.090909,41,0.368421,0.494118,0.422111,85,0.666667,0.300000,0.413793,20,0.394366,0.380952,0.387543,147,0.176471,0.109091,0.134831,55,0.664269,0.488344,0.345789,0.363450,1251,0.642099,0.664269,0.639111,1251
3,1.003400,0.859534,0.822967,0.866499,0.844172,794,0.305263,0.532110,0.387960,109,0.090909,0.024390,0.038462,41,0.423729,0.294118,0.347222,85,0.666667,0.500000,0.571429,20,0.430769,0.380952,0.404332,147,0.300000,0.054545,0.092308,55,0.672262,0.434329,0.378945,0.383698,1251,0.655164,0.672262,0.655150,1251
4,1.003400,0.861746,0.830189,0.886650,0.857491,794,0.396825,0.458716,0.425532,109,0.454545,0.121951,0.192308,41,0.535211,0.447059,0.487179,85,0.631579,0.600000,0.615385,20,0.471698,0.510204,0.490196,147,0.294118,0.090909,0.138889,55,0.710631,0.516309,0.445070,0.458140,1251,0.691208,0.710631,0.694269,1251
5,0.646500,0.853811,0.855000,0.861461,0.858218,794,0.409396,0.559633,0.472868,109,0.416667,0.121951,0.188679,41,0.500000,0.447059,0.472050,85,0.583333,0.700000,0.636364,20,0.442529,0.523810,0.479751,147,0.250000,0.072727,0.112676,55,0.705835,0.493846,0.469520,0.460087,1251,0.698278,0.705835,0.695664,1251


<ipython-input-7-15fbd50be80d>:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(v, collections.MutableMapping):
/data/experiments/raring/miniconda2/envs/semantic_relation/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/experiments/raring/miniconda2/envs/semantic_relation/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/experiments/raring/miniconda2/envs/semantic_relation/lib/python3.9/site-packages/torch/nn/parallel/_fun

Accuracy: 0.7106314948041567


Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'config', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias',

...

Accuracy: 0.6632


,eval_loss,eval_none__precision,eval_none__recall,eval_none__f1-score,eval_none__support,eval_attribution__precision,eval_attribution__recall,eval_attribution__f1-score,eval_attribution__support,eval_causal__precision,eval_causal__recall,eval_causal__f1-score,eval_causal__support,eval_conditional__precision,eval_conditional__recall,eval_conditional__f1-score,eval_conditional__support,eval_contrast__precision,eval_contrast__recall,eval_contrast__f1-score,eval_contrast__support,eval_description__precision,eval_description__recall,eval_description__f1-score,eval_description__support,eval_equivalence__precision,eval_equivalence__recall,eval_equivalence__f1-score,eval_equivalence__support,eval_fulfillment__precision,eval_fulfillment__recall,eval_fulfillment__f1-score,eval_fulfillment__support,eval_identity__precision,eval_identity__recall,eval_identity__f1-score,eval_identity__support,eval_purpose__precision,eval_purpose__recall,eval_purpose__f1-score,eval_purpose__support,eval_summary__precision,eval_summary__recall,eval_summary__f1-score,eval_summary__support,eval_temporal__precision,eval_temporal__recall,eval_temporal__f1-score,eval_temporal__support,eval_accuracy,eval_macro avg__precision,eval_macro avg__recall,eval_macro avg__f1-score,eval_macro avg__support,eval_weighted avg__precision,eval_weighted avg__recall,eval_weighted avg__f1-score,eval_weighted avg__support,eval_runtime,eval_samples_per_second,epoch
fold,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.917879,0.874145,0.804786,0.838033,794.00,0.0,0.0,0.0,6.00,0.364706,0.568807,0.444444,109.00000,0.0,0.0,0.0,14.0,0.000000,0.000000,0.000000,41.00,0.0,0.0,0.0,13.00,0.458333,0.654762,0.539216,84.00000,0.0,0.0,0.0,11.00,0.548387,0.850000,0.666667,20.0,0.0,0.0,0.0,6.00000,0.0,0.0,0.0,6.0,0.361809,0.489796,0.416185,147.00,0.675460,0.217282,0.280679,0.242045,1251.00000,0.668647,0.675460,0.666386,1251.00000,13.014900,96.121000,5.0
2,0.956131,0.889503,0.811083,0.848485,794.00,0.0,0.0,0.0,5.00,0.370968,0.638889,0.469388,108.00000,0.0,0.0,0.0,14.0,0.333333,0.047619,0.083333,42.00,0.0,0.0,0.0,14.00,0.360656,0.523810,0.427184,84.00000,0.0,0.0,0.0,10.00,0.666667,0.700000,0.682927,20.0,0.0,0.0,0.0,6.00000,0.0,0.0,0.0,6.0,0.439791,0.567568,0.495575,148.00,0.685052,0.255076,0.274081,0.250574,1251.00000,0.694682,0.685052,0.680078,1251.00000,13.112200,95.408000,5.0
3,1.042702,0.860403,0.807305,0.833008,794.00,0.0,0.0,0.0,6.00,0.329193,0.490741,0.394052,108.00000,0.0,0.0,0.0,14.0,1.000000,0.024390,0.047619,41.00,0.0,0.0,0.0,13.00,0.426230,0.611765,0.502415,85.00000,0.0,0.0,0.0,11.00,0.454545,0.250000,0.322581,20.0,0.0,0.0,0.0,5.00000,0.0,0.0,0.0,6.0,0.347619,0.496599,0.408964,147.00,0.660000,0.284832,0.223400,0.209053,1250.00000,0.684906,0.660000,0.652155,1250.00000,12.211600,102.362000,5.0
4,0.973943,0.869507,0.823455,0.845855,793.00,0.0,0.0,0.0,6.00,0.312925,0.422018,0.359375,109.00000,0.0,0.0,0.0,14.0,0.000000,0.000000,0.000000,41.00,0.0,0.0,0.0,13.00,0.335714,0.552941,0.417778,85.00000,0.0,0.0,0.0,11.00,1.000000,0.100000,0.181818,20.0,0.0,0.0,0.0,5.00000,0.0,0.0,0.0,6.0,0.385714,0.551020,0.453782,147.00,0.663200,0.241988,0.204120,0.188217,1250.00000,0.663091,0.663200,0.652631,1250.00000,10.381000,120.412000,5.0
avg,0.972664,0.873389,0.811657,0.841345,793.75,0.0,0.0,0.0,5.75,0.344448,0.530114,0.416815,108.50000,0.0,0.0,0.0,14.0,0.333333,0.018002,0.032738,41.25,0.0,0.0,0.0,13.25,0.395233,0.585819,0.471648,84.50000,0.0,0.0,0.0,10.75,0.667400,0.475000,0.463498,20.0,0.0,0.0,0.0,5.50000,0.0,0.0,0.0,6.0,0.383733,0.526246,0.443626,147.25,0.670928,0.249795,0.245570,0.222473,1250.50000,0.677832,0.670928,0.662812,1250.50000,12.179925,103.575750,5.0
std,0.052223,0.012165,0.008280,0.007113,0.50,0.0,0.0,0.0,0.50,0.027929,0.094099,0.049480,0.57735,0.0,0.0,0.0,0.0,0.471405,0.022848,0.040517,0.50,0.0,0.0,0.0,0.50,0.056805,0.058744,0.058853,0.57735,0.0,0.0,0.0,0.50,0.238114,0.357071,0.250751,0.0,0.0,0.0,0.0,0.57735,0.0,0.0,0.0,0.0,0.040543,0.038854,0.039818,0.50,0.011535,0.028128,0.0376

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'config', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias',

Epoch,Training Loss,Validation Loss,None Precision,None Recall,None F1-score,None Support,Causal Precision,Causal Recall,Causal F1-score,Causal Support,Contrast Precision,Contrast Recall,Contrast F1-score,Contrast Support,Equivalence Precision,Equivalence Recall,Equivalence F1-score,Equivalence Support,Identity Precision,Identity Recall,Identity F1-score,Identity Support,Temporal Precision,Temporal Recall,Temporal F1-score,Temporal Support,Others Precision,Others Recall,Others F1-score,Others Support,Accuracy,Macro avg Precision,Macro avg Recall,Macro avg F1-score,Macro avg Support,Weighted avg Precision,Weighted avg Recall,Weighted avg F1-score,Weighted avg Support
1,No log,1.065366,0.715706,0.906801,0.800000,794,0.218750,0.064220,0.099291,109,0.000000,0.000000,0.000000,41,0.216346,0.529412,0.307167,85,0.000000,0.000000,0.000000,20,0.000000,0.000000,0.000000,147,0.666667,0.036364,0.068966,55,0.618705,0.259638,0.219542,0.182203,1251,0.517322,0.618705,0.540308,1251
2,No log,0.963294,0.772414,0.846348,0.807692,794,0.350000,0.192661,0.248521,109,0.307692,0.097561,0.148148,41,0.315789,0.352941,0.333333,85,0.242857,0.850000,0.377778,20,0.387755,0.258503,0.310204,147,0.222222,0.181818,0.200000,55,0.633094,0.371247,0.397119,0.346525,1251,0.611498,0.633094,0.613077,1251


/data/experiments/raring/miniconda2/envs/semantic_relation/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/experiments/raring/miniconda2/envs/semantic_relation/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
